In [ ]:
# Instala o pacote iPython-sql #para usar linguagem SQL
!pip install ipython-sql

In [2]:
import sqlite3
print(sqlite3.sqlite_version)

3.45.1


In [3]:
# Imports
import os #acessar o sistema operacional
import pandas as pd
import sqlite3 #banco de dados

In [4]:
# Define o caminho para o arquivo do banco de dados
arquivo_path = 'novo_banco.db'

In [5]:
# Verifica se o arquivo existe e deleta se existir para criar um novo arquivo depois
if os.path.exists(arquivo_path):
    try:
        os.remove(arquivo_path)
        print(f"Arquivo {arquivo_path} deletado com sucesso!")
    except Exception as e:
        print(f"Erro ao deletar o arquivo {arquivo_path}. Detalhes: {e}")
else:
    print(f"Arquivo {arquivo_path} não encontrado.")

Arquivo novo_banco.db não encontrado.


In [6]:
# Criamos e conectamos a conexão a um banco de dados SQLite
cnn = sqlite3.connect('novo_banco.db')

In [12]:
# Carregamos a extensão SQL
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [13]:
%reload_ext sql

In [18]:
# Definimos o banco de dados
%sql sqlite:///novo_banco.db

Connecting to 'sqlite:///novo_banco.db'

In [8]:
# Carrega o dataset
df = pd.read_csv('dataset.csv')

In [9]:
df.shape #linhas e colunas

(768, 9)

In [10]:
# Copia o dataframe para dentro do banco de dados como uma tabela
df.to_sql('diabetes', cnn)

768

In [16]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,103,30,38,83,43.3,0.183,33,0
1,1,115,70,30,96,34.6,0.529,32,1
2,3,126,88,41,235,39.3,0.704,27,0
3,8,99,84,0,0,35.4,0.388,50,0
4,7,196,90,0,0,39.8,0.451,41,1


In [19]:
%%sql
SELECT * FROM diabetes

Running query in 'sqlite:///novo_banco.db'

index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,103,30,38,83,43.3,0.183,33,0
1,1,115,70,30,96,34.6,0.529,32,1
2,3,126,88,41,235,39.3,0.704,27,0
3,8,99,84,0,0,35.4,0.388,50,0
4,7,196,90,0,0,39.8,0.451,41,1
5,6,148,72,35,0,33.6,0.627,50,1
6,1,85,66,29,0,26.6,0.351,31,0
7,8,183,64,0,0,23.3,0.672,32,1
8,1,89,66,23,94,28.1,0.167,21,0
9,0,137,40,35,168,43.1,2.288,33,1


In [20]:
%%sql
SELECT Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age,
    CASE
        WHEN Age > 50 AND BMI <30 THEN 'Normal' 
        WHEN Age > 50 AND BMI >=30 THEN 'Obeso'
        ELSE '-'
    END as 'Status'
FROM diabetes
WHERE Status IS NOT '-'

Running query in 'sqlite:///novo_banco.db'

Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Status
2,197,70,45,543,30.5,0.158,53,Obeso
8,125,96,0,0,0.0,0.232,54,Normal
10,139,80,0,0,27.1,1.441,57,Normal
1,189,60,23,846,30.1,0.398,59,Obeso
5,166,72,19,175,25.8,0.587,51,Normal
11,143,94,33,146,36.6,0.254,51,Obeso
13,145,82,19,110,22.2,0.245,57,Normal
5,109,75,26,0,36.0,0.546,60,Obeso
4,111,72,47,207,37.1,1.39,56,Obeso
9,171,110,24,240,45.4,0.721,54,Obeso


In [21]:
%%sql
SELECT COUNT(*) FROM(
SELECT Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age,
    CASE
        WHEN Age > 50 AND BMI <30 THEN 'Normal' 
        WHEN Age > 50 AND BMI >=30 THEN 'Obeso'
        ELSE '-'
    END as 'Status'
FROM diabetes
WHERE Status IS NOT '-'
) AS SUBQUERY

Running query in 'sqlite:///novo_banco.db'

COUNT(*)
81


In [22]:
%%sql
CREATE TABLE results (Pregnancies INT,
                        Glucose INT,
                        BloodPressure INT,
                        SkinThickness INT,
                        Insulin INT,
                        BMI DECIMAL(8, 2),
                        DiabetesPedigreeFunction DECIMAL(8, 2),
                        Age INT,
                        Status INT);

Running query in 'sqlite:///novo_banco.db'

++
||
++
++

In [23]:
%%sql
SELECT * FROM results

Running query in 'sqlite:///novo_banco.db'

Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Status


In [24]:
%%sql

INSERT INTO results(Pregnancies, 
                      Glucose, 
                      BloodPressure, 
                      SkinThickness, 
                      Insulin, 
                      BMI, 
                      DiabetesPedigreeFunction, 
                      Age, 
                      Status) 
SELECT Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age,
    CASE
        WHEN Age > 50 AND BMI <30 THEN 'Normal' 
        WHEN Age > 50 AND BMI >=30 THEN 'Obeso'
        ELSE '-'
    END as 'Status'
FROM diabetes
WHERE Status IS NOT '-'

Running query in 'sqlite:///novo_banco.db'

81 rows affected.

++
||
++
++

In [33]:
%%sql
SELECT * FROM results

Running query in 'sqlite:///novo_banco.db'

Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Status
2,197,70,45,543,30.5,0.158,53,Obeso
8,125,96,0,0,0,0.232,54,Normal
10,139,80,0,0,27.1,1.441,57,Normal
1,189,60,23,846,30.1,0.398,59,Obeso
5,166,72,19,175,25.8,0.587,51,Normal
11,143,94,33,146,36.6,0.254,51,Obeso
13,145,82,19,110,22.2,0.245,57,Normal
5,109,75,26,0,36,0.546,60,Obeso
4,111,72,47,207,37.1,1.39,56,Obeso
9,171,110,24,240,45.4,0.721,54,Obeso


In [34]:
m_results = cnn.execute("SELECT * FROM results")

In [35]:
coluns = [coluna[0] for coluna in m_results.description]

In [36]:
m_final = pd.DataFrame.from_records(data = m_results.fetchall(), columns = coluns)

In [37]:
m_final.shape

(81, 9)

In [38]:
m_final.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Status
0,2,197,70,45,543,30.5,0.158,53,Obeso
1,8,125,96,0,0,0.0,0.232,54,Normal
2,10,139,80,0,0,27.1,1.441,57,Normal
3,1,189,60,23,846,30.1,0.398,59,Obeso
4,5,166,72,19,175,25.8,0.587,51,Normal


In [39]:
# Salva em CSV
m_final.to_csv('FINAL.csv', index = False)

In [40]:
#too see each table inside my 
%%sql
SELECT name FROM sqlite_master WHERE type='table';


Running query in 'sqlite:///novo_banco.db'

name
diabetes
results
